In [1]:
from dabry.problem import NavigationProblem, IndexedProblem
from dabry.misc import Utils
from dabry.wind import *
from dabry.model import *
from dabry.penalty import *
from dabry.solver_ef import SolverEF
from dabry.ddf_manager import DDFmanager

$$
\text{penalty} = \frac{A}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2} \frac{(x-c)^2}{\sigma^2}}\\
r_{1/2} = \sigma \sqrt{2 \ln 2}\\
p_{\text{max}} = \frac{A}{\sigma \sqrt{2 \pi}}
$$

In [74]:
pb = IndexedProblem('linear')

f = 1e6
c = f* np.array((0.5, 0.17))
pmax = 1000
rten = f* 0.2
sigma = rten / (2 * np.log(10))

def pf(x):
    return 1 + pmax*np.exp(-0.5*(x - c) @ (x - c)/sigma**2)

penalty = Penalty(pf, length_scale=f)

pb.penalty = penalty

In [75]:
pb.penalty.value(1e6*np.array((0.5, 0.1)))

273.8139993491576

In [76]:
solver = SolverEF(pb, max_steps=500, max_time=50*3600)

In [77]:
res = solver.solve()

Problem | TIMEOPT | 11.41 m/s | 1.00e+06 m | scale 24h20m | orthodromic 24h6m
Steps :    167/500, Extremals :    509, Active :   93, Dist :  47 Comp : 1 Stopped empty active list
Target reached in 22h52m | -6% no wind | -5% orthodromic | cpu time 4.61s


In [78]:
ddf = DDFmanager()
ddf.setup()
ddf.set_case('test_penalty')
ddf.clean_output_dir()
ddf.dump_wind(pb.model.wind, nx=101, ny=101, bl=f*np.array((-0.1, -0.6)), tr=f*np.array((1.1, 0.6)))
ddf.dump_trajs(solver.get_trajs())
ddf.dump_trajs([res.traj], filename='trajectories_timeopt.h5')
ddf.log(pb)